In [2]:
import keras
import csv
import scipy
import numpy
import nltk #from nltk.tokenize import punkt #.tokenize.moses import MosesTokenizer
from gensim.models import KeyedVectors
from itertools import islice
import pickle
import pandas as pd
import re
from nltk.corpus import stopwords
import json
pattern = re.compile(r'[^\s\w_]+')

DATASET_FILE_PATH = "./DataSet/yelp.csv/yelp.csv"
WORD2VEC_FILE_PATH = "./DataSet/FBword2vec/wiki.en.vec"
TRAINING_DATA_PATH = "./DataSet/trainingData"
EXTENDED_DATASET_PATH = "./DataSet/review.json"
AMOUNT_OF_WORDS = 30
FACEBOOK_EMBEDDING_SIZE = 300


C:\Users\skymont\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
C:\Users\skymont\Anaconda3\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [3]:
extended_dataset = pd.read_json(EXTENDED_DATASET_PATH, lines=True)
extended_dataset.head()

,business_id,cool,date,funny,review_id,stars,text,useful,user_id
0,0W4lkclzZThpx3V65bVgig,0,2016-05-28,0,v0i_UHJMo_hPBq9bxWvW4w,5,"Love the staff, love the meat, love the place....",0,bv2nCi5Qv5vroFiqKGopiw
1,AEx2SYEUJmTxVVB18LlCwA,0,2016-05-28,0,vkVSCC7xljjrAI4UGfnKEQ,5,Super simple place but amazing nonetheless. It...,0,bv2nCi5Qv5vroFiqKGopiw
2,VR6GpWIda3SfvPC-lg9H3w,0,2016-05-28,0,n6QzIUObkYshz4dz2QRJTw,5,Small unassuming place that changes their menu...,0,bv2nCi5Qv5vroFiqKGopiw
3,CKC0-MOWMqoeWf6s-szl8g,0,2016-05-28,0,MV3CcKScW05u5LVfF6ok0g,5,Lester's is located in a beautiful neighborhoo...,0,bv2nCi5Qv5vroFiqKGopiw
4,ACFtxLv8pGrrxMm6EgjreA,0,2016-05-28,0,IXvOzsEMYtiJI0CARmj77Q,4,Love coming here. Yes the place always needs t...,0,bv2nCi5Qv5vroFiqKGopiw


In [5]:
labels = extended_dataset[['cool','funny','stars','useful']].copy()
labels.head()

,cool,funny,stars,useful
0,0,0,5,0
1,0,0,5,0
2,0,0,5,0
3,0,0,5,0
4,0,0,4,0


In [6]:
labels.to_pickle("labels.pkl")

In [4]:
extended_dataset.to_pickle("extended_dataframe.pkl")

In [2]:
def load_data_set(path):
    data_set = []
    with open(path, 'r') as csvData:
        data = csv.reader(csvData, delimiter=',')
        for row in islice(data, 1, None):
            data_set.append([row[4], row[3]])
    return data_set

In [2]:


def load_word2vec(path):
    return KeyedVectors.load_word2vec_format(path)




In [3]:
 def vectorize(x,keyedvector):
        try: 
            return pd.Series(keyedvector.word_vec(x,use_norm=False).tolist())
        except KeyError:
            return []#[0]*FACEBOOK_EMBEDDING_SIZE

In [4]:
def text_vectorize(text, keyedvector, stop_words_set):
    
    tokens = nltk.word_tokenize(pattern.sub("",text).lower())
    token_vec = []
    for word in tokens:
        if word not in stop_words_set: token_vec.extend(vectorize(word,keyedvector))
        if len(token_vec) >= FACEBOOK_EMBEDDING_SIZE * AMOUNT_OF_WORDS: break
    
    #padding
    token_vec.extend([0]*(AMOUNT_OF_WORDS*FACEBOOK_EMBEDDING_SIZE-(len(token_vec))))
    
    return pd.Series(
            token_vec,
            index=["word:"+str(i)+"Component:"+str(j) 
                               for i in range(AMOUNT_OF_WORDS) 
                               for j in range(FACEBOOK_EMBEDDING_SIZE)],
            dtype='float16'
        )
        

In [5]:
wordVector = load_word2vec(WORD2VEC_FILE_PATH)

In [6]:
extended_dataset = pd.read_pickle("extended_dataframe.pkl")
extended_dataset = extended_dataset.sample(frac=0.05)

In [ ]:
text_vector = extended_dataset["text"].apply(
    text_vectorize,args=(
        wordVector,
        set(stopwords.words("english"))
    )
)

In [ ]:
text_vector = shape_extended_training_data(extended_dataset,wordVector)

In [1]:
text_vector.head()

NameError: name 'text_vector' is not defined

In [ ]:
text_vector['cool'] = extended_dataset['cool']
text_vector['funny'] = extended_dataset['funny']
text_vector['stars'] = extended_dataset['stars']
text_vector['useful'] = extended_dataset['useful']

In [ ]:
text_vector.to_pickle("5percent30WordsNoStopwordsFacebookVectors.pkl")

In [10]:
def shape_extended_training_data(dataset, keyedvector):

    dataset.drop("business_id",axis=1, inplace=True)
    dataset.drop("date",axis=1, inplace=True)
    dataset.drop("user_id",axis=1, inplace=True)
    
    
    trainingData = []
    #moses = MosesTokenizer()
    for review in dataset['text']:
        
        tokens = nltk.word_tokenize(pattern.sub("",review.lower()))
        
        #Remove stop words
        stop_words = set(stopwords.words("english"))
        
        #Save length of extra aggregation
        wordcount = len(tokens)
        
        #Transforming to vector
        token_vec = []
        for word in tokens:
            if word not in stop_words: token_vec.extend(vectorize(word,keyedvector))
            
            if len(token_vec) >= FACEBOOK_EMBEDDING_SIZE * AMOUNT_OF_WORDS:
                break
        
        token_vec.extend([0]*(AMOUNT_OF_WORDS*FACEBOOK_EMBEDDING_SIZE-(len(token_vec))))
        
        #Shortening
        #tokens = tokens[:AMOUNT_OF_WORDS]
        
        #Adding word_count
        #token_vec.append(wordcount)
        
        #Obtaining label
        #label = review[1]
        #token_vec.append(label)
        
        #Adding to model
        trainingData.append(token_vec)

    #Word headers
    headers = ["word:"+str(i)+"Component:"+str(j) for i in range(AMOUNT_OF_WORDS) for j in range(FACEBOOK_EMBEDDING_SIZE) ]
    #Word count header
    #headers.append('wordcount')
    #Label
    #headers.append('label')
    return pd.DataFrame(trainingData,columns=headers)


In [38]:
#Might be valuable to remove stopwords in the future
def shape_training_data(dataset, keyedvector):

    trainingData = []
    #moses = MosesTokenizer()
    for review in dataSet:
        
        tokens = nltk.word_tokenize(pattern.sub("",str(review[0])).lower())
        
        #Remove stop words
        stop_words = set(stopwords.words("english"))
        
        #Save length of extra aggregation
        wordcount = len(tokens)
        
        #Transforming to vector
        token_vec = []
        for word in tokens:
            if word not in stop_words: token_vec.extend(vectorize(word,keyedvector))
            
            if len(token_vec) >= FACEBOOK_EMBEDDING_SIZE * AMOUNT_OF_WORDS:
                break
        
        token_vec.extend([0]*(AMOUNT_OF_WORDS*FACEBOOK_EMBEDDING_SIZE-(len(token_vec))))
        
        #Shortening
        #tokens = tokens[:AMOUNT_OF_WORDS]
        
        #Adding word_count
        token_vec.append(wordcount)
        
        #Obtaining label
        label = review[1]
        token_vec.append(label)
        
        #Adding to model
        trainingData.append(token_vec)

    #Word headers
    headers = ["word:"+str(i)+"Component:"+str(j) for i in range(AMOUNT_OF_WORDS) for j in range(FACEBOOK_EMBEDDING_SIZE) ]
    #Word count header
    headers.append('wordcount')
    #Label
    headers.append('label')
    return pd.DataFrame(trainingData,columns=headers)




In [39]:
trainingData = shape_training_data(dataSet,wordVector)
trainingData.head()
trainingData.to_pickle("10_words_NoStopWords_FBvec_300_dataframe.pkl")

In [40]:
#trainingData.columns = ["word:"+str(i)+"Component:"+str(j) for i in range(AMOUNT_OF_WORDS) for j in range(FACEBOOK_EMBEDDING_SIZE)]+["wordcount","label"]
trainingData.head()
#trainingData.to_pickle("200_words_FBvec_300_dataframe.pkl")

,word:0Component:0,word:0Component:1,word:0Component:2,word:0Component:3,word:0Component:4,word:0Component:5,word:0Component:6,word:0Component:7,word:0Component:8,word:0Component:9,...,word:9Component:292,word:9Component:293,word:9Component:294,word:9Component:295,word:9Component:296,word:9Component:297,word:9Component:298,word:9Component:299,wordcount,label
0,-0.021861,0.147650,-0.17284,-0.007612,-0.118620,-0.046999,0.194030,-0.32793,0.051490,0.535680,...,0.10594,0.13861,-0.07890,-0.145350,-0.063092,0.10283,-0.053264,0.321260,155,5
1,-0.059007,0.121840,-0.19363,0.248770,-0.068540,0.095495,-0.250840,-0.26307,-0.057524,0.410680,...,0.11442,-0.29999,0.15141,-0.100010,-0.187290,0.12408,0.393330,0.014831,257,5
2,0.197970,-0.091041,-0.11557,-0.031160,-0.380090,0.289140,0.103970,0.32546,0.086976,-0.040133,...,0.00000,0.00000,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000,15,4
3,0.387090,0.164850,-0.26387,0.122690,-0.298180,-0.522280,0.044023,0.34574,-0.025861,0.087658,...,-0.30717,-0.23020,0.59489,-0.149490,-0.101980,0.46842,0.305260,0.167270,76,5
4,0.120340,-0.085165,0.17255,0.058870,0.008439,0.071639,0.118450,-0.24949,0.221510,-0.067038,...,0.13121,-0.21514,0.38700,0.002831,-0.269890,0.11523,0.373380,-0.084600,84,5


In [41]:
### TRAINING MODEL

In [54]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.layers import SpatialDropout1D
from keras.layers import Dropout
from sklearn.model_selection import train_test_split
import numpy as np
# fix random seed for reproducibility
seed = 42
np.random.seed(seed)

In [55]:
### Splitting train and test sets

In [56]:
dataset = pd.read_pickle("10_words_NoStopWords_FBvec_300_dataframe.pkl") 
y = dataset.pop("label")
X = dataset

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=428)

In [57]:
### Defining model

In [58]:
model = Sequential()
#model.add(Embedding(top_words, 32, input_length=max_words))
model.add(Conv1D(input_shape=(X_train.shape[1],1),filters=8, kernel_size=10, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
SpatialDropout1D(0.8)
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.8))
model.add(Dense(1))
model.compile(loss='mean_absolute_percentage_error', optimizer='adam', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_6 (Conv1D)            (None, 3001, 8)           88        
_________________________________________________________________
max_pooling1d_6 (MaxPooling1 (None, 1500, 8)           0         
_________________________________________________________________
flatten_6 (Flatten)          (None, 12000)             0         
_________________________________________________________________
dense_11 (Dense)             (None, 128)               1536128   
_________________________________________________________________
dropout_6 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_12 (Dense)             (None, 1)                 129       
Total params: 1,536,345
Trainable params: 1,536,345
Non-trainable params: 0
_________________________________________________________________


In [59]:
### Input Reshape for Neural Net

In [60]:
X_train_shape = X_train.shape
X_trainMAT = X_train.values.astype(np.float16).reshape(X_train_shape[0], X_train_shape[1], 1)
X_test_shape = X_test.shape
X_testMAT = X_test.values.astype(np.float16).reshape(X_test_shape[0], X_test_shape[1], 1) 

In [61]:

model.fit(X_trainMAT,
          y_train,
          validation_data=(X_testMAT, y_test), epochs=40, batch_size=128, verbose=2)

Train on 8000 samples, validate on 2000 samples
Epoch 1/40
 - 1s - loss: 57.0182 - acc: 0.1520 - val_loss: 43.9219 - val_acc: 0.1345
Epoch 2/40
 - 1s - loss: 51.6494 - acc: 0.1660 - val_loss: 42.0541 - val_acc: 0.1580
Epoch 3/40
 - 1s - loss: 49.7762 - acc: 0.1700 - val_loss: 41.0745 - val_acc: 0.1600
Epoch 4/40
 - 1s - loss: 47.6138 - acc: 0.1816 - val_loss: 40.3886 - val_acc: 0.1580
Epoch 5/40
 - 1s - loss: 46.5382 - acc: 0.1849 - val_loss: 39.8737 - val_acc: 0.1645
Epoch 6/40
 - 1s - loss: 46.1177 - acc: 0.1771 - val_loss: 39.2632 - val_acc: 0.1710
Epoch 7/40
 - 1s - loss: 44.3218 - acc: 0.1850 - val_loss: 37.4181 - val_acc: 0.2315
Epoch 8/40
 - 1s - loss: 44.8171 - acc: 0.1885 - val_loss: 39.3670 - val_acc: 0.1730
Epoch 9/40
 - 1s - loss: 44.1267 - acc: 0.1883 - val_loss: 38.5378 - val_acc: 0.1865
Epoch 10/40
 - 1s - loss: 41.8284 - acc: 0.2045 - val_loss: 37.3542 - val_acc: 0.2070
Epoch 11/40
 - 1s - loss: 41.2160 - acc: 0.2009 - val_loss: 37.8636 - val_acc: 0.1930
Epoch 12/40
 - 

In [ ]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

In [8]:
X_train

,word:0Component:0,word:0Component:1,word:0Component:2,word:0Component:3,word:0Component:4,word:0Component:5,word:0Component:6,word:0Component:7,word:0Component:8,word:0Component:9,...,word:199Component:291,word:199Component:292,word:199Component:293,word:199Component:294,word:199Component:295,word:199Component:296,word:199Component:297,word:199Component:298,word:199Component:299,wordcount
9254,-0.076770,-0.115590,-0.298870,0.328810,0.036217,0.046055,-0.132970,-0.232200,-0.212600,0.228990,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,54
1561,-0.400830,0.139920,0.129070,-0.118440,-0.244090,-0.071496,-0.126240,-0.105380,0.021209,0.175540,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,68
1670,-0.241220,-0.052791,-0.002987,0.216260,-0.200030,-0.070722,0.185170,0.095463,0.313940,0.311030,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,108
6087,-0.400830,0.139920,0.129070,-0.118440,-0.244090,-0.071496,-0.126240,-0.105380,0.021209,0.175540,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,28
6669,-0.199640,0.085116,-0.055189,0.131420,-0.034263,0.116870,0.024297,-0.365910,-0.160210,0.130410,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,37
5933,-0.336600,0.138890,-0.068331,0.007915,-0.020023,0.093429,0.101820,-0.215100,-0.188440,0.151480,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,95
8829,-0.065334,-0.093031,-0.017571,0.200070,0.029521,-0.039920,-0.163280,-0.072946,0.089604,0.080907,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,77
7945,0.001160,0.191540,0.004549,0.303440,0.259600,0.286950,0.271440,-0.293130,0.287860,0.509090,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,48
3508,-0.148770,-0.014936,0.091772,0.146630,-0.449550,-0.022379,0.069493,-0.329310,-0.236200,0.123390,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,71
2002,0.057781,-0.036046,-0.063841,0.546840,-0.439060,-0.088080,-0.252800,-0.457690,-0.186750,0.080070,...,0.072315,0.060320,0.010248,0.091444,-0.146000,-0.136400,0.133910,0.009458,-0.038994,319
